In [1]:
import os
import sys
sys.path.append('../../src')
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
from datetime import datetime
import time
from hydra import initialize, compose
import pathlib
# import config

import data_process.neg_sample as ng_sample
from data_process.utils import mix_merge
from data_process.data_split import data_split_user
from evaluate_entity import CustomHR, CustomNDCG, CustomRoc, CustomRoctop, CustomRecall_top, CustomPrecision_top
from model_entity import EntityCat_num
from data_utils import CatData
from utils.constants import DEFAULT_USER_COL,DEFAULT_ITEM_COL,DEFAULT_RATING_COL, DEFAULT_TIMESTAMP_COL


from sklearn import metrics, preprocessing
from sklearn.metrics import roc_auc_score, average_precision_score, recall_score, precision_score

import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

# import argparse
torch.manual_seed(0)

/Users/hao/.pyenv/versions/3.6.15/envs/torch-cpu5/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from ignite.engine import Engine, Events, create_supervised_trainer, create_supervised_evaluator, RemovableEventHandle
from ignite.metrics import Accuracy, Loss, Metric
from ignite.handlers import ModelCheckpoint, EarlyStopping
from ignite.exceptions import NotComputableError
from ignite.metrics.metric import sync_all_reduce, reinit__is_reduced
from ignite.contrib.handlers.tqdm_logger import ProgressBar
# from ignite.contrib.handlers import TensorboardLogger 
from ignite.contrib.handlers.wandb_logger import *

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [4]:
with initialize(version_base=None, config_path="../conf"):
    cfg = compose(config_name="config", overrides=[])

In [5]:
if device.type =='cpu':
    BATCH_SIZE = cfg.params.batch_size_cpu
    EPOCHS  = cfg.params.epochs_cpu
else:
    BATCH_SIZE = cfg.params.batch_size_gpu
    EPOCHS  = cfg.params.epochs_gpu

In [6]:
if device.type == 'cpu':
    use_amp=False
    df_train_pos  = ng_sample.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_pos))
    df_train_neg = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_neg))
    df_test_ori = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.test)).iloc[:202,]
    df_all_features = pd.read_csv(pathlib.Path(cfg.path.root, cfg.file.all_features))
    df_train_pos = df_train_pos.sort_values(by=[DEFAULT_USER_COL]).iloc[:100,].reset_index(drop=True)
    df_train_neg = df_train_neg.sort_values(by=[DEFAULT_USER_COL]).iloc[:100*cfg.params.neg_train,].reset_index(drop=True)
else:
    use_amp=True
    df_train_pos  = ng_sample.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_pos))
    df_train_neg = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_neg))
    df_test_ori = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.test))
    df_all_features = pd.read_csv(pathlib.Path(cfg.path.root, cfg.file.all_features))
    df_train_pos = df_train_pos.sort_values(by=[DEFAULT_USER_COL]).reset_index(drop=True)
    df_train_neg = df_train_neg.sort_values(by=[DEFAULT_USER_COL]).reset_index(drop=True)

In [7]:
df_train_pos[DEFAULT_RATING_COL] = 1

In [8]:
def concat_index(df1, df2):
    df2.index = df2.index//cfg.params.neg_train
    return pd.concat([df1, df2], axis=0).sort_index(kind='mregesort').reset_index(drop=True)

In [9]:
df_train_all = concat_index(df_train_pos, df_train_neg)

In [10]:
df_train_all['flag'] = 1
df_test_ori['flag'] = 0
df_all = pd.concat([df_train_all, df_test_ori], axis=0).reset_index(drop=True)

user features: 
       'WindowID_user', 'Split', 'City',
       'State', 'Country', 'Zip_user', 'DegreeType', 'Major', 'GraduationDate',
       'WorkHistoryCount', 'TotalYearsExperience', 'CurrentlyEmployed',
       'ManagedOthers', 'ManagedHowMany',
       
job features: 
       'WindowID_job', 'City_job',
       'State_job', 'Country_job', 'Zip_job', 'StartDate', 'EndDate',

# Adding the Sbert embedding

In [11]:
job_title_emb = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.title_emb))

In [12]:
job_title_emb.rename(columns={'item_id':DEFAULT_ITEM_COL, 'Title_em':'title_em'}, inplace=True)

In [13]:
title_em = pd.merge(df_all, job_title_emb[['title_em', 'itemid']], how='left', on=[DEFAULT_ITEM_COL]).title_em

In [14]:
title_em.shape

(702,)

### Choose the features and process data for the training

In [15]:
user_features = ['City']
user_features_extend = [DEFAULT_USER_COL] + user_features

item_features = ['City_job']
item_features_extend =[DEFAULT_ITEM_COL] + item_features

base_features = [DEFAULT_USER_COL, DEFAULT_ITEM_COL, DEFAULT_RATING_COL]

In [16]:
df_mix_merge = mix_merge(df_all , df_all_features, user_features_extend, item_features_extend)

In [17]:
df_mix_merge.columns

Index(['userid', 'City', 'itemid', 'City_job', 'rating'], dtype='object')

In [18]:
def _cat_encode(df_data, list_f, encoder):
    for f in list_f:
        df_data[f] = encoder.fit_transform(df_data[f].astype('category').cat.codes.values)
    return df_data

In [19]:
def _embedding_dimension(df_all_encode, features_to_train, max_dim=50):

    embedding_size = []
    features_to_em = [i for i in features_to_train if i !=DEFAULT_RATING_COL]
    for c in features_to_em:
        num_unique_values = int(df_all_encode[c].nunique())
        embed_dim = int(min(np.ceil(num_unique_values/2), max_dim))
        embedding_size.append([num_unique_values, embed_dim])  
    return embedding_size

In [20]:
def encode_data(df_mix_merge, features_to_code, features_to_train, title_em, max_dim=50):
    encoder = preprocessing.LabelEncoder()
#     features_to_code = df_mix_merge.columns
    df_all_encode = _cat_encode(df_mix_merge, features_to_code, encoder)
    embedding_size = _embedding_dimension(df_all_encode, features_to_train, max_dim)
#     print('hao1---', df_all_encode.shape)
#     print('hao2---', title_em.shape)
    df_all_encode['title_em']=title_em
    df_train = df_all_encode[df_all.flag==1]
    df_test = df_all_encode[df_all.flag==0]
    df_train = df_train[features_to_train+['title_em']]
    df_test = df_test[features_to_train+['title_em']]
#     embedding_size = _embedding_dimension(df_all_encode, features_to_train, max_dim)
    return df_train, df_test, embedding_size

In [21]:
num_feature=[]
features_to_code = df_mix_merge.columns
features_to_train = [DEFAULT_USER_COL, DEFAULT_ITEM_COL]+ user_features + item_features+[DEFAULT_RATING_COL]


In [22]:

df_train,  df_test, embedding_size = encode_data(df_mix_merge, features_to_code, features_to_train, title_em, max_dim=cfg.params.emb_dim)

print(f'The size of embedding layers:{embedding_size}')

The size of embedding layers:[[36, 18], [413, 50], [35, 18], [230, 50]]


In [23]:
features_to_train = [DEFAULT_USER_COL, DEFAULT_ITEM_COL]+ user_features + item_features+['title_em']+[DEFAULT_RATING_COL]
df_train= df_train[features_to_train]
df_test= df_test[features_to_train]

In [24]:
df_train

,userid,itemid,City,City_job,title_em,rating
0,0,93,26,167,"[-0.037161242, 0.029026292, -0.013159127, 0.03...",1
1,0,164,26,1,"[0.03175809, -0.04611786, -0.05164853, -0.0086...",0
2,0,176,26,52,"[-0.024634995, -0.043930512, 0.0149936555, 0.0...",0
3,0,114,26,92,"[-0.037364177, 0.030169426, -0.006370666, 0.03...",0
4,0,353,26,53,"[0.016816339, -0.03030617, -0.008112824, -0.05...",0
...,...,...,...,...,...,...
495,35,121,18,118,"[0.029377155, 0.093792565, 0.0025794103, 0.048...",1
496,35,164,18,1,"[0.03175809, -0.04611786, -0.05164853, -0.0086...",0
497,35,176,18,52,"[-0.024634995, -0.043930512, 0.0149936555, 0.0...",0
498,35,113,18,148,"[0.031220712, -0.007913577, -0.031209802, -0.0...",0


In [25]:
df_train_split, df_val_split = data_split_user(df_train, val_size=0.2)

np_train = df_train_split.values
np_val = df_val_split.values
np_test = df_test.values

In [26]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [27]:
train_dataset = CatData(np_train)
val_dataset = CatData(np_val)
test_dataset = CatData(np_test) 
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0,  worker_init_fn=seed_worker,generator=g)
val_loader = data.DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False, num_workers=0,  worker_init_fn=seed_worker,generator=g)
test_loader = data.DataLoader(test_dataset, batch_size=cfg.params.neg_test+1, shuffle=False, num_workers=0,worker_init_fn=seed_worker,generator=g )

In [48]:
type(next(iter(train_dataset))[0])

numpy.ndarray

In [47]:
next(iter(train_dataset))

(array([0, 93, 26, 167,
        array([-3.71612422e-02,  2.90262923e-02, -1.31591270e-02,  3.04880459e-02,
        -2.68472247e-02,  5.00881970e-02, -1.32076321e-02,  3.17287492e-03,
        -5.26629947e-03,  5.15368767e-02,  2.07514428e-02,  2.98920125e-02,
         4.21630517e-02,  5.98949008e-02,  6.80278018e-02,  2.67940853e-02,
         1.76773109e-02, -1.31420670e-02,  7.41418377e-02, -3.68065909e-02,
        -9.06064082e-03, -3.62051232e-03, -4.04246487e-02,  3.65843214e-02,
         4.80788201e-03, -1.88543908e-02,  3.20609212e-02, -5.24251023e-03,
        -6.14058226e-02,  2.03530863e-02,  2.04773936e-02, -5.09758182e-02,
         4.55549918e-02,  1.26327416e-02,  2.26326824e-06, -3.89049156e-03,
         1.82351824e-02, -2.13731211e-02, -8.42611268e-02, -1.45234121e-02,
        -5.42238764e-02, -3.19802128e-02, -6.65268442e-03,  4.20785323e-02,
        -5.36166131e-03, -9.26883593e-02,  6.18459657e-02,  1.35155125e-02,
        -4.65138778e-02, -4.33438309e-02, -1.08021563e-02

In [28]:
model = EntityCat_num(embedding_size = embedding_size, num_numerical_cols = len(num_feature),
               output_size = 2)
model.to(device)

EntityCat_num(
  (all_embeddings): ModuleList(
    (0): Embedding(36, 18)
    (1): Embedding(413, 50)
    (2): Embedding(35, 18)
    (3): Embedding(230, 50)
  )
  (embedding_dropout): Dropout(p=0.4, inplace=False)
  (layers): Sequential(
    (0): Linear(in_features=136, out_features=100, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.4, inplace=False)
    (3): Linear(in_features=100, out_features=2, bias=True)
  )
)

In [29]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=cfg.params.lr)

In [30]:
print(model)

EntityCat_num(
  (all_embeddings): ModuleList(
    (0): Embedding(36, 18)
    (1): Embedding(413, 50)
    (2): Embedding(35, 18)
    (3): Embedding(230, 50)
  )
  (embedding_dropout): Dropout(p=0.4, inplace=False)
  (layers): Sequential(
    (0): Linear(in_features=136, out_features=100, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.4, inplace=False)
    (3): Linear(in_features=100, out_features=2, bias=True)
  )
)


## Model the training 

In [31]:
def output_trans_loss(output):
    return output['y_pred'], output['label']

val_metrics_train = {
    'auc': CustomRoc(),
    "loss": Loss(criterion, output_transform=output_trans_loss)
}

val_metrics_test = {
    'hr': CustomHR(),
    'ndcg': CustomNDCG(),
    'auc': CustomRoc(),
    'roc_top': CustomRoctop(),
    'recall_top': CustomRecall_top(threshold=0.5),
    'precision_top': CustomPrecision_top(threshold=0.5),
    "loss": Loss(criterion, output_transform=output_trans_loss)
}

In [32]:
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
def train_step(engine, batch):
    model.train()
    optimizer.zero_grad()
    x, y = batch[0].to(device), batch[1].to(device)
    with torch.cuda.amp.autocast(enabled=use_amp):
        y_pred = model(x)[:,1]
        loss = criterion(y_pred, y.float())
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
    return loss.item()

def validation_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, label = batch[0].to(device), batch[1].to(device)
        y_pred = model(x)[:,1]
        label=label.float()
        return {'label':label, 'y_pred':y_pred}

def test_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, label = batch[0].to(device), batch[1].to(device)
        y_pred = model(x)[:,1]
        label=label.float()
        y_pred_top, indices = torch.topk(y_pred, engine.state.topk)
        y_pred_top = y_pred_top.detach().cpu().numpy()
        reco_item = torch.take(x[:,1], indices).cpu().numpy().tolist()
        pos_item = x[0,1].cpu().numpy().tolist()  # ground truth, item id
        label_top = label[indices].cpu().numpy()
        return {'pos_item':pos_item, 'reco_item':reco_item, 'y_pred_top':y_pred_top, 'label_top':label_top, 'label':label, 'y_pred':y_pred}

trainer = Engine(train_step)

train_evaluator = Engine(validation_step)
# train_evaluator.state_dict_user_keys.append('topk')

val_evaluator = Engine(validation_step)
# val_evaluator.state_dict_user_keys.append('topk')

test_evaluator = Engine(test_step)
test_evaluator.state_dict_user_keys.append('topk')

# @val_evaluator.on(Events.STARTED)
# def init_user_value():
#     val_evaluator.state.topk=3
    
# @train_evaluator.on(Events.STARTED)
# def init_user_value():
#     train_evaluator.state.topk=3

@train_evaluator.on(Events.STARTED)
def init_user_value():
    test_evaluator.state.topk=cfg.params.topk
    
    
# Attach metrics to the evaluators
for name, metric in val_metrics_train.items():
    metric.attach(train_evaluator, name)

for name, metric in val_metrics_train.items():
    metric.attach(val_evaluator, name)

    
for name, metric in val_metrics_test.items():
    metric.attach(test_evaluator, name)
    
# Eearly_stop 
def score_function(engine):
    val_loss = engine.state.metrics['auc']
    return val_loss

Eearly_stop_handler = EarlyStopping(patience=cfg.params.patience, score_function=score_function, trainer=trainer)
val_evaluator.add_event_handler(Events.COMPLETED, Eearly_stop_handler)

In [33]:
@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    train_evaluator.run(train_loader)
    metrics = train_evaluator.state.metrics
    auc = metrics['auc']
    loss = metrics['loss']
    print(f'Training Results- Epoch[{trainer.state.epoch}]  Avg loss: {loss:.2f} \
          Avg auc:{auc:.2f}')


@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    val_evaluator.run(val_loader)
    metrics = val_evaluator.state.metrics
    auc = metrics['auc']
    loss = metrics['loss']
    print(f'Validation Results- Epoch[{trainer.state.epoch}]  Avg loss: {loss:.2f} \
          Avg auc:{auc:.2f}')

    
@trainer.on(Events.COMPLETED)
def log_test_results(trainer):
    test_evaluator.run(test_loader)
    metrics = test_evaluator.state.metrics
    hr = metrics['hr']
    ndcg = metrics['ndcg']
    auc = metrics['auc']
    roc_top = metrics['roc_top']
    recall = metrics['recall_top']
    precision = metrics['precision_top']
    loss = metrics['loss']
    print(f"Test Results - Epoch[{trainer.state.epoch}]  Avg loss: {loss:.2f} \
     Avg ndcg: {ndcg:.2f}  Avg auc: {auc:.2f}  Avg auc_top: {roc_top:.2f} \
      Avg recall: {recall:.2f}  Avg precision: {precision:.2f}")

pbar = ProgressBar(persist=False)
# pbar.attach(trainer)
pbar.attach(trainer)
# from ignite.handlers import Timer
# timer = Timer(average=True)
# timer.attach(trainer,
#              start=Events.STARTED,
#              resume=Events.EPOCH_STARTED,
#              pause=Events.EPOCH_COMPLETED,
#              step=Events.EPOCH_COMPLETED)

In [37]:
next(iter(train_loader))

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found object

In [34]:
trainer.run(train_loader, max_epochs=EPOCHS)

Current run is terminating due to exception: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found object
Engine run is terminating due to exception: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found object


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found object

In [ ]:
config_dict = dict(cfg.params)
config_dict['Features']='-'.join(user_features+item_features)
wandb_logger = WandBLogger(
    project="pytorch-jrs",
    name="-".join(user_features)+'-'+'-'.join(item_features),
    config=config_dict,
    tags=["entity", "jrs"]
)

to_save = {'model': model}
checkpoint_handler = ModelCheckpoint(
    wandb_logger.run.dir,
    n_saved=2, filename_prefix='best',
    score_name="auc",
    global_step_transform=global_step_from_engine(trainer)
)

val_evaluator.add_event_handler(Events.COMPLETED, checkpoint_handler, to_save)

    
wandb_logger.attach_output_handler(
    trainer,
    event_name=Events.ITERATION_COMPLETED,
    tag="training",
    output_transform=lambda loss: {"loss": loss}
)

wandb_logger.attach_output_handler(
    train_evaluator,
    event_name=Events.EPOCH_COMPLETED,
    tag="training",
    metric_names=['loss','auc'],
    global_step_transform=lambda *_: trainer.state.iteration,
)

wandb_logger.attach_output_handler(
    val_evaluator,
    event_name=Events.EPOCH_COMPLETED,
    tag="validation",
    metric_names=['loss',"auc"],
    global_step_transform=lambda *_: trainer.state.iteration,
)


wandb_logger.attach_output_handler(
    test_evaluator,
    event_name=Events.COMPLETED,
    tag="test",
    metric_names=['loss',"auc", 'hr', 'ndcg', 'roc_top', 'recall_top', 'precision_top'],
    global_step_transform=lambda *_: trainer.state.iteration,
)


wandb_logger.attach_opt_params_handler(
    trainer,
    event_name=Events.ITERATION_STARTED,
    optimizer=optimizer,
    param_name='lr'  # optional
)

# wandb_logger.watch(model) 

# trainer.run(train_loader, max_epochs=EPOCHS)
wandb_logger.close()